# Georgia Tech Summer 2025 MSA Practicum Project

### 2. Data Retrieval

**Hosted on [Hypertrial.ai](https://www.hypertrial.ai/university-projects/georgia-tech)**

---

In [4]:
import pandas as pd
import requests
from io import StringIO

## 📊 Bitcoin Historical Data (Coin Metrics)

As part of the Stacking Sats project, you’ll be working with a daily historical dataset of Bitcoin metrics provided by [Coin Metrics](https://coinmetrics.io/). This dataset is widely used in industry and research to understand Bitcoin’s market dynamics, and it's updated daily through a public GitHub repository.

You’ll use this data to develop, backtest, and evaluate Bitcoin accumulation strategies over multiple years. 

### 📁 Dataset Information
- **Source**: Coin Metrics Community Data
- **URL**: [btc.csv](https://raw.githubusercontent.com/coinmetrics/data/master/csv/btc.csv)
- **License**: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
- **Update Frequency**: Daily

This CSV file contains a wide range of metrics. You're encouraged to explore them all to enhance your strategy design.

In [6]:
# Coin Metrics BTC CSV (raw GitHub URL)
url = "https://raw.githubusercontent.com/coinmetrics/data/master/csv/btc.csv"

# Download the content
response = requests.get(url)
response.raise_for_status()  # raises an error for bad responses

# Parse CSV content
btc_df = pd.read_csv(StringIO(response.text))

# Show the df
btc_df

/var/folders/zn/3r5r5myd1dq34m6jv1g5rq7m0000gn/T/ipykernel_75277/4280389743.py:9: DtypeWarning: Columns (147) have mixed types. Specify dtype option on import or set low_memory=False.
  btc_df = pd.read_csv(StringIO(response.text))


,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd
0,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,2025-05-16,829195.0,11288.0,2967924.0,19945177.0,897.0,531723.0,9313183.0,45.0,100198.0,...,1.209042e+10,0.285095,29526.191905,0.001000,103.566269,4.422050,0.025915,0.018481,103786.43,coinbase-btc-usd-spot
5978,2025-05-17,740377.0,11286.0,2968069.0,19927428.0,896.0,531733.0,9309379.0,45.0,100204.0,...,6.423317e+09,0.176797,18243.669854,0.000824,85.055423,4.416293,0.025909,0.018574,103500.03,coinbase-btc-usd-spot
5979,2025-05-18,601421.0,11281.0,2967962.0,19936453.0,896.0,531717.0,9310144.0,45.0,100206.0,...,7.600407e+09,0.129617,13741.441247,0.000609,64.609672,4.417651,0.025950,0.018786,103163.87,coinbase-btc-usd-spot
5980,2025-05-19,718046.0,11283.0,2967667.0,19943914.0,896.0,531661.0,9311741.0,46.0,100222.0,...,1.515012e+10,0.256888,27131.472643,0.001013,106.940251,4.424386,0.025910,0.018900,106476.34,coinbase-btc-usd-spot


Read the data spec [here](https://gitbook-docs.coinmetrics.io/network-data/network-data-overview/addresses/address-balances?utm_source=chatgpt.com)